<a href="https://colab.research.google.com/github/abkerper/project_voting/blob/main/Connor_Merge_Attempt_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
! git clone https://github.com/DS3001/project_voting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split

fatal: destination path 'project_voting' already exists and is not an empty directory.


In [3]:
data_df = pd.read_csv("./project_voting/data/voting_VA.csv")
data_df.head()

,Unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,11161,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,AL GORE,DEMOCRAT,5092,11925,20220315,TOTAL
1,11162,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,6352,11925,20220315,TOTAL
2,11163,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,RALPH NADER,GREEN,220,11925,20220315,TOTAL
3,11164,2000,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,OTHER,OTHER,261,11925,20220315,TOTAL
4,11165,2000,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,AL GORE,DEMOCRAT,16255,36846,20220315,TOTAL


In [6]:
data_2000 = data_df[(data_df['year'] == 2000)]
data_2004 = data_df[(data_df['year'] == 2004)]
data_2008 = data_df[(data_df['year'] == 2008)]
data_2012 = data_df[(data_df['year'] == 2012)]
data_2016 = data_df[(data_df['year'] == 2016)]
data_2020 = data_df[(data_df['year'] == 2020)]
data_2008.head()

,Unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
938,30189,2008,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,BARACK OBAMA,DEMOCRAT,7607,15623,20220315,TOTAL
939,30190,2008,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,JOHN MCCAIN,REPUBLICAN,7833,15623,20220315,TOTAL
940,30191,2008,VIRGINIA,VA,ACCOMACK,51001,US PRESIDENT,OTHER,OTHER,183,15623,20220315,TOTAL
941,30192,2008,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,BARACK OBAMA,DEMOCRAT,29792,50984,20220315,TOTAL
942,30193,2008,VIRGINIA,VA,ALBEMARLE,51003,US PRESIDENT,JOHN MCCAIN,REPUBLICAN,20576,50984,20220315,TOTAL


In [8]:
demo_2008 = pd.read_csv("/content/project_voting/data/county_data/0002_ds176_20105_county_E.csv", encoding='latin1')
#This is the dataset that is a monster

row = demo_2008.iloc[0]  # Row 0 contains the descriptions of the variables

# Prints out each cell to show variables but honestly easier to just look at the codebook
print("Variables in Dataset:")

Variables in Dataset:


<ipython-input-8-f1f58fc645a4>:1: DtypeWarning: Columns (3,4,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,2

In [9]:
demo_2008 = demo_2008[demo_2008["STUSAB"] == "VA"] # Sorts the df for just VA counties. Can change if we want to look at US as a whole
demo_2008.head()

demo_2008= demo_2008.loc[:,("COUNTY","JMAE001","JMBE002","JMBE003","JMBE004","JMBE005",
                     "JMBE006","JMBE007","JMBE008","JQBE001","JL0E001","JL0E002",
                     "JL0E003")]  #Selects variables we want in the clean df

new_column_names = {"JMAE001" : "Total Population",
        "JMBE002":     "White alone",
        "JMBE003":    "Black or African American alone",
        "JMBE004":     "American Indian and Alaska Native alone",
        "JMBE005":     "Asian alone",
        "JMBE006":     "Native Hawaiian and Other Pacific Islander alone",
        "JMBE007":     "Some other race alone",
        "JMBE008":     "Two or more races",
        "JQBE001":     "Per Capita income in the past 12 months",
        "JL0E001":     "Median age: Total",
        "JL0E002":     "Median age: Male",
        "JL0E003":     "Median age: Female",
}  #These are all taken from the codebook in the github. MAKE SURE YOU'RE LOOKING AT THE RIGHT CODEBOOK FOR THE RIGHT CSV. I didnt realize that until way too late.

demo_2008.rename(columns=new_column_names, inplace=True)
demo_2008.head()

,COUNTY,Total Population,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races,Per Capita income in the past 12 months,Median age: Total,Median age: Male,Median age: Female
2821,Accomack County,34066,22639,9729,30,166,0,702,800,22766.0,43.9,41.5,45.8
2822,Albemarle County,96633,79493,9563,168,4146,28,1164,2071,36685.0,37.6,36.3,38.9
2823,Alleghany County,16406,15250,979,3,81,0,18,75,22013.0,44.7,43.2,46.0
2824,Amelia County,12517,9228,2954,31,47,18,125,114,24197.0,41.5,40.6,42.7
2825,Amherst County,32315,24762,6201,301,149,0,310,592,21097.0,41.1,39.9,42.2


In [38]:
data_by_year = [data_2000, data_2004, data_2008, data_2012, data_2016, data_2020]
df_wins = pd.DataFrame()

for year in data_by_year:
    wins = pd.DataFrame()
    county_names = year['county_name'].values
    county_names = list(set(county_names))
    for county in county_names:
        county_rows = year[year['county_name'] == county]
        max_votes = county_rows['candidatevotes'].idxmax()
        winning_row = year.loc[max_votes]
        wins = pd.concat([wins, pd.DataFrame([winning_row])], ignore_index=True)
    df_wins = pd.concat([df_wins, wins], ignore_index=True)

data_2000 = df_wins[df_wins['year'] == 2000]
data_2004 = df_wins[df_wins['year'] == 2004]
data_2008 = df_wins[df_wins['year'] == 2008]
data_2012 = df_wins[df_wins['year'] == 2012]
data_2016 = df_wins[df_wins['year'] == 2016]
data_2020 = df_wins[df_wins['year'] == 2020]

In [39]:
print(type(data_2008))


<class 'pandas.core.frame.DataFrame'>


In [51]:
data_2008 = data_2008.rename(columns={'county_name': 'COUNTY'})
data_2008['COUNTY'] = data_2008['COUNTY'].str.lower()
data_2008.head()

demo_2008['COUNTY'] = demo_2008['COUNTY'].str.lower()
demo_2008.head()

demo_2008['COUNTY'] = demo_2008['COUNTY'].str.replace(' ', '')
data_2008['COUNTY'] = data_2008['COUNTY'].str.replace(' ', '')

demo_2008['COUNTY'] = demo_2008['COUNTY'].str.strip()
data_2008['COUNTY'] = data_2008['COUNTY'].str.strip()
merged_df = pd.merge(data_2008, demo_2008, on="COUNTY")
merged_df.head()

,Unnamed: 0,year,state,state_po,COUNTY,county_fips,office,candidate,party,candidatevotes,...,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races,Per Capita income in the past 12 months,Median age: Total,Median age: Male,Median age: Female
